In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.preprocessing import StandardScaler, Imputer, FunctionTransformer, LabelBinarizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
files = [f for f in listdir('./') if '.csv' in f]
files

['test_df_block13_open26.csv',
 'test_df_block13_open52.csv',
 'test_df_block13_open78.csv',
 'test_df_block26_open26.csv',
 'test_df_block26_open52.csv',
 'test_df_block26_open78.csv',
 'test_df_block52_open26.csv',
 'test_df_block52_open52.csv',
 'test_df_block52_open78.csv',
 'train_df_block13_open26.csv',
 'train_df_block13_open52.csv',
 'train_df_block13_open78.csv',
 'train_df_block26_open26.csv',
 'train_df_block26_open52.csv',
 'train_df_block26_open78.csv',
 'train_df_block52_open26.csv',
 'train_df_block52_open52.csv',
 'train_df_block52_open78.csv']

# Build 9 models for different combination of block weeks = [13, 26, 52] and open weeks = [26, 52, 78] with voting classifier

In [12]:
file = open("results.txt", "w+")
file.close()

block = ['13', '26', '52']
openw = ['26', '52', '78']
for b in block:
    for o in openw:
        train = './train_df_block'+b+'_open'+o+'.csv'
        test = './test_df_block'+b+'_open'+o+'.csv'
        print(train, test)
        df_train = pd.read_csv(train)
        df_test = pd.read_csv(test)
        unwanted = ['id', 'name', 'first_review', 'last_review', 'permanently_closed']
        wanted = [i for i in df_train.columns if i not in unwanted]
        X_train = df_train[wanted]
        X_test = df_test[wanted]
        ytrain = df_train['permanently_closed']
        ytest = df_test['permanently_closed']
        imputer = Imputer(missing_values=np.nan, strategy='mean', axis=1)
        ss = StandardScaler() 
        pipe = make_pipeline(imputer, ss) 
        Xtrain = pipe.fit_transform(X_train)
        Xtest = pipe.transform(X_test)

        list_of_classifiers = [
            ('logreg', LogisticRegression()),
        #     ('sgd_cls', SGDClassifier()),
        #     ('dt', DecisionTreeClassifier()),
            ('rf', RandomForestClassifier()),
            ('gb', GradientBoostingClassifier())
        ]
        vclf = VotingClassifier(list_of_classifiers, voting='soft')
        params = {
            'logreg__C': [0.1, 1.0, 10.0],
            'logreg__penalty': ['l1', 'l2'],
        #     'sgd_cls__loss':['log'], #, 'squared_loss'],
        #     'sgd_cls__penalty':['elasticnet'],
        #     'sgd_cls__alpha':np.logspace(-1,1,5),
        #     'sgd_cls__l1_ratio':[i/10.0 for i in range(6)],
            #'dt__max_depth': [3, 5, 7, 9],
            'rf__max_depth': [5, 7],
            'rf__n_estimators': [100, 500],
            'gb__max_depth': [5, 7],
            'gb__n_estimators': [100, 500],
        }
        gs = GridSearchCV(vclf, params, verbose=2)    
        gs.fit(Xtrain, ytrain)
        with open("results.txt","a+") as file:
            file.write(train+'\t'+test+'\n')            
            file.write('score: '+str(gs.score(Xtest, ytest))+'\n')            
            file.write('best parameters:'+'\n')
            file.write(str(gs.best_params_))
            file.write('\n')

./train_df_block13_open26.csv ./test_df_block13_open26.csv
Fitting 3 folds for each of 96 candidates, totalling 288 fits
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   0.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   0.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   0.9s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   1.6s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   1.7s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=5

[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   1.8s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   1.8s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100, total=   1.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=100, total=   1.0s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=1

[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100, total=   1.1s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100, total=   1.2s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500, total=   1.7s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500, total=   1.7s
[CV] gb__max_depth=5, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l2, rf__max_depth=7, rf__n_esti

...

[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500, total=   2.2s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=500, total=   2.2s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   1.5s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   1.5s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=1.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=1

[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=100, total=   2.1s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=100 
[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=100, total=   2.2s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   2.5s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500 
[CV]  gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_estimators=500, total=   2.6s
[CV] gb__max_depth=7, gb__n_estimators=100, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=7, rf__n_esti

[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=500, total=   3.1s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=5, rf__n_estimators=500, total=   3.4s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100 
[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100, total=   2.8s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100 
[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=100, total=   2.3s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=0.1, logreg__penalty=l2, rf__max_depth=7, rf__n_estimators=1

[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   3.0s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100 
[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=100, total=   3.1s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   4.0s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500 
[CV]  gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_estimators=500, total=   3.7s
[CV] gb__max_depth=7, gb__n_estimators=500, logreg__C=10.0, logreg__penalty=l1, rf__max_depth=5, rf__n_esti

[Parallel(n_jobs=1)]: Done 288 out of 288 | elapsed: 11.8min finished


# Read in results.txt

In [38]:
lines = [line.strip() for line in open("./results.txt")]
D = {'block_weeks': [], 'open_weeks': [], 'score': [], 'gb__max_depth': [], 'gb__n_estimators': [], 
     'logreg__C': [], 'logreg__penalty': [], 'rf__max_depth': [], 'rf__n_estimators': []}

for i, line in enumerate(lines):
    print(line.strip())
    if i%4 == 3:
        print()
    if i%12 == 11:
        print()
    
    if i%4 == 0:        
        D['block_weeks'].append(int(line[16:18]))
        D['open_weeks'].append(int(line[23:25]))
        #print(line[16:18])
        #print(line[23:25])
    if i%4 == 1:        
        D['score'].append(float(line[7:]))
        #print(float(line[7:]))
    if i%4 == 3:        
        D['gb__max_depth'].append(int(parameters[0].split(': ')[1]))
        D['gb__n_estimators'].append(int(parameters[1].split(': ')[1]))
        D['logreg__C'].append(float(parameters[2].split(': ')[1]))
        D['logreg__penalty'].append(str(parameters[3].split(': ')[1]))
        D['rf__max_depth'].append(int(parameters[4].split(': ')[1]))
        D['rf__n_estimators'].append(int(parameters[5].split(': ')[1]))
        
        #parameters = line[1:-1].split(', ')
        #print(int(parameters[1].split(': ')[1]))
        

./train_df_block13_open26.csv	./test_df_block13_open26.csv
score: 0.857638888889
best parameters:
{'gb__max_depth': 7, 'gb__n_estimators': 500, 'logreg__C': 1.0, 'logreg__penalty': 'l1', 'rf__max_depth': 5, 'rf__n_estimators': 100}

./train_df_block13_open52.csv	./test_df_block13_open52.csv
score: 0.864583333333
best parameters:
{'gb__max_depth': 5, 'gb__n_estimators': 500, 'logreg__C': 1.0, 'logreg__penalty': 'l1', 'rf__max_depth': 7, 'rf__n_estimators': 100}

./train_df_block13_open78.csv	./test_df_block13_open78.csv
score: 0.861111111111
best parameters:
{'gb__max_depth': 7, 'gb__n_estimators': 500, 'logreg__C': 0.1, 'logreg__penalty': 'l1', 'rf__max_depth': 5, 'rf__n_estimators': 500}


./train_df_block26_open26.csv	./test_df_block26_open26.csv
score: 0.850694444444
best parameters:
{'gb__max_depth': 7, 'gb__n_estimators': 100, 'logreg__C': 1.0, 'logreg__penalty': 'l1', 'rf__max_depth': 5, 'rf__n_estimators': 500}

./train_df_block26_open52.csv	./test_df_block26_open52.csv
score: 0

# Score and best parameters for each model

In [51]:
results = pd.DataFrame(D).transpose()
results.reindex(['block_weeks', 'open_weeks', 'score', 'logreg__C', 'logreg__penalty', 
                 'rf__max_depth', 'rf__n_estimators','gb__max_depth', 'gb__n_estimators'])

,0,1,2,3,4,5,6,7,8
block_weeks,13,13,13,26,26,26,52,52,52
open_weeks,26,52,78,26,52,78,26,52,78
score,0.857639,0.864583,0.861111,0.850694,0.871528,0.864583,0.868056,0.868056,0.854167
logreg__C,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
logreg__penalty,'l1','l1','l1','l1','l1','l1','l1','l1','l1'
rf__max_depth,7,7,7,7,7,7,7,7,7
rf__n_estimators,100,100,100,100,100,100,100,100,100
gb__max_depth,5,5,5,5,5,5,5,5,5
gb__n_estimators,500,500,500,500,500,500,500,500,500


Block = 13

In [20]:
(0.857638888889 + 0.864583333333 + 0.861111111111)/3

0.861111111111

Block = 26

In [21]:
(0.850694444444 + 0.871527777778 + 0.864583333333)/3

0.8622685185183333

Block = 52

In [22]:
(0.868055555556 + 0.868055555556 + 0.854166666667)/3

0.8634259259263333

# Rerun each model to print accuracy and classification report 

In [78]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

block = ['13', '26', '52']
openw = ['26', '52', '78']
for b in block:
    accuracy = []
    for o in openw:
        train = './train_df_block'+b+'_open'+o+'.csv'
        test = './test_df_block'+b+'_open'+o+'.csv'
        print('block weeks: ', b, '\topen weeks: ', o)
        df_train = pd.read_csv(train)
        df_test = pd.read_csv(test)
        unwanted = ['id', 'name', 'first_review', 'last_review', 'permanently_closed']
        wanted = [i for i in df_train.columns if i not in unwanted]
        X_train = df_train[wanted]
        X_test = df_test[wanted]
        ytrain = df_train['permanently_closed']
        ytest = df_test['permanently_closed']
        imputer = Imputer(missing_values=np.nan, strategy='mean', axis=1)
        ss = StandardScaler() 
        pipe = make_pipeline(imputer, ss) 
        Xtrain = pipe.fit_transform(X_train)
        Xtest = pipe.transform(X_test)

        list_of_classifiers = [
            ('logreg', LogisticRegression(C=0.1, penalty='l1')),
            ('rf', RandomForestClassifier(max_depth=7, n_estimators=100)),
            ('gb', GradientBoostingClassifier(max_depth=5, n_estimators=500))
        ]
        vclf = VotingClassifier(list_of_classifiers, voting='soft')
        vclf.fit(Xtrain, ytrain)
        ypred = vclf.predict(Xtest)
        acc = accuracy_score(ytest, ypred)
        accuracy.append(acc)
        confusion = np.array(confusion_matrix(ytest, ypred))
        print('accuracy: ', acc)
        print('confusion matrix: ')
        print(confusion)
        print(classification_report(ytest, ypred))
    print('Mean accuracy for '+b+' weeks: ', sum(accuracy)/float(len(accuracy)))
    print()
    print()

block weeks:  13 	open weeks:  26
accuracy:  0.857638888889
confusion matrix: 
[[201   9]
 [ 32  46]]
             precision    recall  f1-score   support

          0       0.86      0.96      0.91       210
          1       0.84      0.59      0.69        78

avg / total       0.86      0.86      0.85       288

block weeks:  13 	open weeks:  52
accuracy:  0.868055555556
confusion matrix: 
[[199  11]
 [ 27  51]]
             precision    recall  f1-score   support

          0       0.88      0.95      0.91       210
          1       0.82      0.65      0.73        78

avg / total       0.86      0.87      0.86       288

block weeks:  13 	open weeks:  78
accuracy:  0.864583333333
confusion matrix: 
[[202   8]
 [ 31  47]]
             precision    recall  f1-score   support

          0       0.87      0.96      0.91       210
          1       0.85      0.60      0.71        78

avg / total       0.86      0.86      0.86       288

Mean accuracy for 13 weeks:  0.863425925926


blo

# Around 30 closed restaurants are predicted to be open acroos all models:

In [77]:
count = 0
for p, t in zip(ypred, list(ytest)):
    if t == 1 and p == 0:
        count += 1
count

33